# Link ISDN with their Sub-ICB-Locations

## Plain English summary

In this notebook we create a dataframe of Sub-Integrated Care Board Locations (SICBL), the Integrated Care Boards (ICB) that contain them, the Integrated Stroke Delivery Networks (ISDN) that they belong to. The resulting DataFrame is saved to file.

The areas in each ISDN are shown on a map that has some inconsistencies. In a previous document (["ISDN boundaries"](https://github.com/samuel-book/geography_data/blob/mt_catchment/geojson/isdn_boundaries.ipynb)), we decided that the ISDN geography was best defined as a collection of ICBs (2022 data). We will combine this ICB-ISDN lookup table with a lookup table linking SICBL to ICB.


## Data sources

+ The SICBL to ICB file is downloaded from the Open Geography Portal and converted from Excel .xlsx to .csv. It is the 2022 data.
+ ICB-ISDN lookup: defined in a previous document (["ISDN boundaries"](https://github.com/samuel-book/geography_data/blob/mt_catchment/geojson/isdn_boundaries.ipynb)).

## Notebook setup

In [1]:
import numpy as np
# For handling the tabular data:
import pandas as pd

# For tracking paths to files:
import os
from dataclasses import dataclass

In [2]:
# Define file paths
@dataclass(frozen=True)
class Paths:
    '''Singleton object for storing paths to data and database.'''

    dir_tabular_ons = '../data_tabular/ons_data'
    dir_output = 'output_data'

    df_sicbl_icb = 'LOC22_ICB22_NHSER22_EN_LU.csv'

    df_output = 'SICBL_ICB_ISDN.csv'

paths = Paths()

## Load reference data table

The SICBL are called "LOC" in the column headings.

In [3]:
df_sicbl_icb = pd.read_csv(
    os.path.join(paths.dir_tabular_ons, paths.df_sicbl_icb), index_col=0)

df_sicbl_icb.head()

,LOC22CDH,LOC22NM,ICB22CD,ICB22CDH,ICB22NM,NHSER22CD,NHSER22CDH,NHSER22NM
LOC22CD,,,,,,,,
E38000006,02P,NHS South Yorkshire ICB - 02P,E54000061,QF7,NHS South Yorkshire Integrated Care Board,E40000012,Y63,North East and Yorkshire
E38000007,99E,NHS Mid and South Essex ICB - 99E,E54000026,QH8,NHS Mid and South Essex Integrated Care Board,E40000007,Y61,East of England
E38000008,02Q,NHS Nottingham and Nottinghamshire ICB - 02Q,E54000060,QT1,NHS Nottingham and Nottinghamshire Integrated ...,E40000011,Y60,Midlands
E38000014,00Q,NHS Lancashire and South Cumbria ICB - 00Q,E54000048,QE1,NHS Lancashire and South Cumbria Integrated Ca...,E40000010,Y62,North West
E38000015,00R,NHS Lancashire and South Cumbria ICB - 00R,E54000048,QE1,NHS Lancashire and South Cumbria Integrated Ca...,E40000010,Y62,North West


## Link ICB to ISDN

Manually assign an ISDN to these regions:

In [4]:
isdn_dict = {
    'NHS Cheshire and Merseyside Integrated Care Board': 'Cheshire and Merseyside',
    'NHS Staffordshire and Stoke-on-Trent Integrated Care Board': 'North Midlands',
    'NHS Shropshire, Telford and Wrekin Integrated Care Board': 'North Midlands',
    'NHS Lincolnshire Integrated Care Board': 'East Midlands',
    'NHS Leicester, Leicestershire and Rutland Integrated Care Board': 'East Midlands',
    'NHS Coventry and Warwickshire Integrated Care Board': 'West Midlands',
    'NHS Herefordshire and Worcestershire Integrated Care Board': 'West Midlands',
    'NHS Norfolk and Waveney Integrated Care Board': 'East of England (North)',
    'NHS Suffolk and North East Essex Integrated Care Board': 'East of England (North)',
    'NHS Bedfordshire, Luton and Milton Keynes Integrated Care Board': 'East of England (South)',
    'NHS Hertfordshire and West Essex Integrated Care Board': 'East of England (South)',
    'NHS Mid and South Essex Integrated Care Board': 'East of England (South)',
    'NHS North West London Integrated Care Board': 'London',
    'NHS North Central London Integrated Care Board': 'London',
    'NHS North East London Integrated Care Board': 'London',
    'NHS South East London Integrated Care Board': 'London',
    'NHS South West London Integrated Care Board': 'London',
    'NHS Kent and Medway Integrated Care Board': 'Kent and Medway',
    'NHS Frimley Integrated Care Board': 'Frimley ICS',
    'NHS Cornwall and the Isles of Scilly Integrated Care Board': 'Devon, Cornwall and Isles of Scilly',
    'NHS Devon Integrated Care Board': 'Devon, Cornwall and Isles of Scilly',
    'NHS Somerset Integrated Care Board': 'Gloucester, BSW, BNSSG and Somerset',
    'NHS Bristol, North Somerset and South Gloucestershire Integrated Care Board': 'Gloucester, BSW, BNSSG and Somerset',
    'NHS Bath and North East Somerset, Swindon and Wiltshire Integrated Care Board': 'Gloucester, BSW, BNSSG and Somerset',
    'NHS Dorset Integrated Care Board': 'Wessex',
    'NHS Hampshire and Isle of Wight Integrated Care Board': 'Wessex',
    'NHS Gloucestershire Integrated Care Board': 'Gloucester, BSW, BNSSG and Somerset',
    'NHS Buckinghamshire, Oxfordshire and Berkshire West Integrated Care Board': 'Thames Valley',
    'NHS Lancashire and South Cumbria Integrated Care Board': 'Lancashire and South Cumbria',
    'NHS North East and North Cumbria Integrated Care Board': 'North East and Cumbria',
    'NHS Humber and North Yorkshire Integrated Care Board': 'Humber Coast and Vale',
    'NHS Surrey Heartlands Integrated Care Board': 'Frimley ICS',
    'NHS Sussex Integrated Care Board': 'Sussex',
    'NHS West Yorkshire Integrated Care Board': 'West Yorkshire and Harrogate',
    'NHS Birmingham and Solihull Integrated Care Board': 'West Midlands',
    'NHS Cambridgeshire and Peterborough Integrated Care Board': 'East of England (South)',
    'NHS Greater Manchester Integrated Care Board': 'Greater Manchester',
    'NHS Derby and Derbyshire Integrated Care Board': 'East Midlands',
    'NHS Northamptonshire Integrated Care Board': 'East Midlands',
    'NHS Nottingham and Nottinghamshire Integrated Care Board': 'East Midlands',
    'NHS South Yorkshire Integrated Care Board': 'South Yorkshire',
    'NHS Black Country Integrated Care Board': 'North Midlands'
}

# Convert region-to-ISDN dictionary to a Series:
isdn_series = pd.Series(isdn_dict, name='ISDN')
isdn_series.index.name='ICB22NM'

## Link the two tables

The SICBL-ICB dataframe and the ICB-ISDN series only have one column in common: the ICB names, `ICB22NM`. Merge the two sets of data by this shared column:

In [5]:
df_sicbl_icb_isdn = pd.merge(
    df_sicbl_icb,
    isdn_series,
    left_on='ICB22NM',
    right_index=True,
    how='left'
)

Sort the values so that nearby SICBL appear near each other in the list:

In [6]:
df_sicbl_icb_isdn = df_sicbl_icb_isdn.sort_values(
    ['NHSER22CD', 'ISDN', 'ICB22NM', 'LOC22NM'])

View the resulting dataframe:

In [7]:
df_sicbl_icb_isdn

,LOC22CDH,LOC22NM,ICB22CD,ICB22CDH,ICB22NM,NHSER22CD,NHSER22CDH,NHSER22NM,ISDN
LOC22CD,,,,,,,,,
E38000240,93C,NHS North Central London ICB - 93C,E54000028,QMJ,NHS North Central London Integrated Care Board,E40000003,Y56,London,London
E38000255,A3A8R,NHS North East London ICB - A3A8R,E54000029,QMF,NHS North East London Integrated Care Board,E40000003,Y56,London,London
E38000256,W2U3Z,NHS North West London ICB - W2U3Z,E54000027,QRV,NHS North West London Integrated Care Board,E40000003,Y56,London,London
E38000244,72Q,NHS South East London ICB - 72Q,E54000030,QKK,NHS South East London Integrated Care Board,E40000003,Y56,London,London
E38000245,36L,NHS South West London ICB - 36L,E54000031,QWE,NHS South West London Integrated Care Board,E40000003,Y56,London,London
...,...,...,...,...,...,...,...,...,...
E38000025,02T,NHS West Yorkshire ICB - 02T,E54000054,QWO,NHS West Yorkshire Integrated Care Board,E40000012,Y63,North East and Yorkshire,West Yorkshire and Harrogate
E38000190,03R,NHS West Yorkshire ICB - 03R,E54000054,QWO,NHS West Yorkshire Integrated Care Board,E40000012,Y63,North East and Yorkshire,West Yorkshire and Harrogate
E38000225,15F,NHS West Yorkshire ICB - 15F,E54000054,QWO,NHS West Yorkshire Integrated Care Board,E40000012,Y63,North East and Yorkshire,West Yorkshire and Harrogate


Save this dataframe to file:

In [8]:
output_path = os.path.join(paths.dir_output, paths.df_output)

print(f'Save data to here: {output_path}')

df_sicbl_icb_isdn.to_csv(output_path)

Save data to here: output_data/SICBL_ICB_ISDN.csv
